In [ ]:
import requests
import bs4
import numpy as np
import pandas as pd
from unidecode import unidecode
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

import time
from datetime import datetime

In [ ]:
def scroll_page_down(driver,wait):

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(wait)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [ ]:
link = 'https://imobiliariarazao.com.br/busca.php?termoPesquisa=Curitiba&codCity=3314&tipoNegocio=2&isLancamento=0&tipo_Imovel%5B%5D=3&referencia=&endereco='

In [ ]:
def get_last_page(LINK):

    print('Abrindo site')
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(LINK)
    driver.maximize_window()

    print('Indo para o fim da página primeira página')
    scroll_page_down(driver,wait=0.5)

    print('Esperando o botão ser carregado')
    wait = WebDriverWait(driver, 50)
    wait.until(EC.presence_of_element_located((By.XPATH,'//*[@id="resultados"]/section/div/div[2]/div/ul/li[5]/a')))

    time.sleep(3)

    print('Encontrando o botão da última página e clicando')
    ultima_pagina = driver.find_element(By.XPATH,'//*[@id="resultados"]/section/div/div[2]/div/ul/li[5]/a')
    ultima_pagina.click()

    print("Esperando o botão ser carregado")
    wait = WebDriverWait(driver, 50)
    wait.until(EC.presence_of_element_located((By.XPATH,'//*[@id="resultados"]/section/div/div[2]/div/ul/li[1]/a')))  

    print('Coletando o número total de páginas')
    soup = bs4.BeautifulSoup(driver.page_source, 'html.parser')
    n_ultima_pagina = int(soup.find_all('span',{'class':'btn-padr active'})[0].getText())

    driver.quit()

    return n_ultima_pagina

In [ ]:
def get_last_page(LINK,chrome_options):

    print('Abrindo site')
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(LINK)
    driver.maximize_window()

    print('Indo para o fim da página primeira página')
    scroll_page_down(driver,wait=0.5)

    print('Esperando o botão ser carregado')
    wait = WebDriverWait(driver, 50)
    wait.until(EC.presence_of_element_located((By.XPATH,'//*[@id="resultados"]/section/div/div[2]/div/ul/li[5]/a')))

    time.sleep(3)

    print('Encontrando o botão da última página e clicando')
    ultima_pagina = driver.find_element(By.XPATH,'//*[@id="resultados"]/section/div/div[2]/div/ul/li[5]/a')
    ultima_pagina.click()

    print("Esperando o botão ser carregado")
    wait = WebDriverWait(driver, 50)
    wait.until(EC.presence_of_element_located((By.XPATH,'//*[@id="resultados"]/section/div/div[2]/div/ul/li[1]/a')))  

    print('Coletando o número total de páginas\n')
    soup = bs4.BeautifulSoup(driver.page_source, 'html.parser')
    n_ultima_pagina = int(soup.find_all('span',{'class':'btn-padr active'})[0].getText())

    driver.quit()

    return n_ultima_pagina

def wait_load_button(driver):
    print('Esperando o botão carregar...')
    wait = WebDriverWait(driver, 50)
    wait.until(EC.presence_of_element_located((By.XPATH,'//*[@id="resultados"]/section/div/div[2]/div/ul/li[1]/a')))  

    return None


def get_number_page(driver, soup):

    print('Coletando o número da página atual')
    n_pagina_atual = int(soup.find_all('span',{'class':'btn-padr active'})[0].getText())
    print(f'Pagina Atual: {n_pagina_atual}')

    return n_pagina_atual

def pass_next_page(driver, pagina_atual, n_pagina_atual, n_ultima_pagina):

    print('Selecionando e clicando na próxima página\n')
    if pagina_atual <= 3:
        page_pass = driver.find_element(By.XPATH,f'//*[@id="resultados"]/section/div/div[2]/div/ul/li[{2+pagina_atual}]/a')
        page_pass.click()
    elif n_pagina_atual > n_ultima_pagina-2:
        page_pass = driver.find_element(By.XPATH,f'//*[@id="resultados"]/section/div/div[2]/div/ul/li[{int(6-(n_ultima_pagina - n_pagina_atual))}]/a')
        page_pass.click()
    else:
        page_pass = driver.find_element(By.XPATH,'//*[@id="resultados"]/section/div/div[2]/div/ul/li[5]/a')
        page_pass.click()
    
    return driver


def get_link_anuncios(n_ultima_pagina,chrome_options):

    lista_de_links = []

    print('Abrindo a primeira página novamente')
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(link)
    driver.maximize_window()

    for pagina_atual in range(1, n_ultima_pagina):

        scroll_page_down(driver,wait=0.5)

        wait_load_button(driver)

        soup = bs4.BeautifulSoup(driver.page_source, 'html.parser')

        n_pagina_atual = get_number_page(driver,soup)

        print('Coletando links dos anúncios')
        lista_de_links = lista_de_links + [ anuncio['onclick'].split("'")[1] for anuncio in soup.find_all('button',{'class':'btn btn-padr btn-detalhes detalhes'})]

        wait_load_button(driver)

        driver = pass_next_page(driver, pagina_atual, n_pagina_atual, n_ultima_pagina)
    
    return lista_de_links


import time

chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-dev-shm-usage')
LINK = 'https://imobiliariarazao.com.br/busca.php?termoPesquisa=Curitiba&codCity=3314&tipoNegocio=2&isLancamento=0&tipo_Imovel%5B%5D=3&referencia=&endereco='

n_ultima_pagina = get_last_page(LINK,chrome_options)

lista_de_links = get_link_anuncios(n_ultima_pagina,chrome_options)

Abrindo site
Indo para o fim da página primeira página
Esperando o botão ser carregado
Encontrando o botão da última página e clicando
Esperando o botão ser carregado
Coletando o número total de páginas

Abrindo a primeira página novamente
Esperando o botão carregar...
Coletando o número da página atual
Pagina Atual: 1
Coletando links dos anúncios
Esperando o botão carregar...
Selecionando e clicando na próxima página

Esperando o botão carregar...
Coletando o número da página atual
Pagina Atual: 2
Coletando links dos anúncios
Esperando o botão carregar...
Selecionando e clicando na próxima página

Esperando o botão carregar...
Coletando o número da página atual
Pagina Atual: 3
Coletando links dos anúncios
Esperando o botão carregar...
Selecionando e clicando na próxima página

Esperando o botão carregar...
Coletando o número da página atual
Pagina Atual: 4
Coletando links dos anúncios
Esperando o botão carregar...
Selecionando e clicando na próxima página

Esperando o botão carregar..

In [ ]:
base_url = 'https://imobiliariarazao.com.br/'
len(lista_de_links)

585

In [ ]:
def coalesce(value):
    try:
        value
    except:
        return np.nan
    return value

def try_get_value(dict_, key):
    try:
        return dict_[key]
    except:
        return np.nan

In [ ]:
def get_info_anuncios(lista_de_links):

    anuncios_list = []
    base_url = 'https://imobiliariarazao.com.br/'

    for anuncio in lista_de_links:

        link = base_url + anuncio
        print(link)
        res = requests.get(link)
        soup = bs4.BeautifulSoup(res.content,'html.parser')

        anuncios_aux = {}

        # site
        anuncios_aux['site'] = 'razao'

        # link
        anuncios_aux['data_coleta'] = datetime.today().strftime("%Y-%m-%d")

        # link
        anuncios_aux['link'] = anuncio

        # titulo
        anuncios_aux['titulo'] = unidecode(coalesce(soup.find_all('h1',{'class':'titleFicha'})[0].getText()))

        # endereco
        anuncios_aux['endereco'] = unidecode(coalesce(soup.find_all('div',{'class':'enderecoImovel enderecoFicha'})[0].find('p').getText().strip()))

        # condominio
        anuncios_aux['condominio'] = coalesce(soup.find_all('p',{'class':'valorCond'})[0].getText())

        # iptu
        anuncios_aux['iptu'] = coalesce(soup.find_all('p',{'class':'valorIptu'})[0].getText())

        # aluguel
        anuncios_aux['aluguel'] = coalesce(soup.find_all('p',{'class':'valorPrincipalImovel'})[0].getText())

        # itens_do_imovel
        anuncios_aux['itens_imovel'] = unidecode(coalesce(', '.join([i.getText() for i in soup.find_all('div',{'class':'col-sm-6 itensDescricao'})])))

        # descrição
        anuncios_aux['descricao'] = unidecode(coalesce(soup.find_all('div',{'class','observaFicha'})[0].find('p').getText()))

        # atributos
        try:
            anuncios_aux['atributos'] = [item.find('p',{'class':'quantItem'}).getText() for item in soup.find_all('div',{'class':'itensImovel'})]
        except:
            anuncios_aux['atributos'] = np.nan

        anuncios_list.append(anuncios_aux)

    return pd.DataFrame(anuncios_list)


https://imobiliariarazao.com.br/ficha.php?url=alugar-apartamento-centro-curitiba-01255006
https://imobiliariarazao.com.br/ficha.php?url=alugar-apartamento-2-quartos-centro-curitiba-02485006
https://imobiliariarazao.com.br/ficha.php?url=alugar-apartamento-1-quarto-centro-curitiba-02622001
https://imobiliariarazao.com.br/ficha.php?url=alugar-apartamento-3-quartos-tingui-curitiba-02053001
https://imobiliariarazao.com.br/ficha.php?url=alugar-apartamento-1-quarto-centro-curitiba-03572007
https://imobiliariarazao.com.br/ficha.php?url=alugar-apartamento-3-quartos-boa-vista-curitiba-03593001
https://imobiliariarazao.com.br/ficha.php?url=alugar-apartamento-1-quarto-reboucas-curitiba-03007003
https://imobiliariarazao.com.br/ficha.php?url=alugar-apartamento-3-quartos-bigorrilho-curitiba-03674001
https://imobiliariarazao.com.br/ficha.php?url=alugar-apartamento-1-quarto-centro-curitiba-03789002
https://imobiliariarazao.com.br/ficha.php?url=alugar-apartamento-3-quartos-cidade-industrial-curitiba-017

In [ ]:
anuncios = pd.DataFrame(anuncios_list)

## Tratamento de Dados

In [ ]:
anuncios

,titulo,endereco,condominio,iptu,aluguel,itens_imovel,descricao,atributos
0,Apartamento no CENTRO de 98 m2,"AL DOUTOR CARLOS DE CARVALHO, no 816 - Centro ...","Cond. R$ 1422,00","IPTU R$ 91,75",R$ 2.300,"Andar: 6.o, Aptos por andar: 3, Central de gas...","Ponto de Referencia: Au Au Lanches, Hospital S...","[(--), 1, 1, 1, 98 m²]"
1,Apartamento no Centro de 84 m2 com 2 quartos,"R MARIANO TORRES, no 401 - Centro - Curitiba",Cond. R$ 815,IPTU R$ 122.02,R$ 2.600,"Aptos por andar: 4, BWC Social: 1, Central de ...",Pontos de Referencia: Mercadorama rua XV e ao ...,"[2, (--), 1, 1, 84 m²]"
2,Apartamento no Centro de 48.6 m2 com 1 quarto,"R CONSELHEIRO LAURINDO, no 781 - Centro - Curi...",Cond. R$ 545,IPTU R$ 60.87,R$ 1.650,"Aptos por andar: 6, BWC Social: 1, Central de ...","Pontos de Referencia: Terminal do Guadalupe, ...","[1, (--), 1, 1, 48.6 m²]"
3,Apartamento no Tingui de 70.81 m2 com 3 quartos,"R NICOLAU SALOMAO, no 550 - Tingui - Curitiba",Cond. R$ 460,IPTU R$ 53.87,R$ 1.700,"Aptos por andar: 4, BWC Social: 1, Condominio ...","Pontos de Referencia: Unidade de Saude Tingui,...","[3, (--), 1, 1, 70.81 m²]"
4,Apartamento no Centro de 106.31 m2 com 1 quarto,"R JOAO NEGRAO, no 805 - Centro - Curitiba",,IPTU R$ 197.09,R$ 1.500,"Aptos por andar: 2, BWC Social: 1, Central de ...","Pontos de Referencia: Shopping Estacao, UTFPR ...","[1, (--), 1, (--), 106.31 m²]"
...,...,...,...,...,...,...,...,...
580,Apartamento no PortAGo de 46 mAe com 1 quarto,"Rua CapitAGo Tenente Maris de Barros, nAs 206 ...",Cond. R$ 240,IPTU R$ 65.53,R$ 1.500,"Andar: 3, BWC Social: 1, Churrasqueiras: 1, Co...","Apartamento para locaASSAGo em Curitiba, no ba...","[1, (--), 1, (--), 46 mÂē]"
581,Apartamento no Bigorrilho de 104 m2 com 3 quartos,"Rua Padre Anchieta, no 2690 - Bigorrilho - Cur...",Cond. R$ 944,IPTU R$ 0.01,R$ 3.000,"Andar: 2, BWC Social: 2, Churrasqueiras: 1, Co...",Apartamento tem:3 quartos (sendo um deles suit...,"[3, 1, 3, (--), 104 m²]"
582,Apartamento no Bigorrilho de 188 m2 com 3 quartos,"Rua General Aristides Athayde Junior, no 1355 ...",Cond. R$ 1282.6,IPTU R$ 263.37,R$ 5.000,"Andar: 3, BWC Social: 2, Condominio: Edificio ...","Apartamento para Locacao, 3 quartos sendo um s...","[3, 1, 3, 2, 188 m²]"
583,Apartamento no JUVEVE de 110 m2 com 3 quartos,"R MANOEL EUFRASIO, no 695 - Juveve - Curitiba","Cond. R$ 580,00","IPTU R$ 125,01",R$ 2.000,"Andar: 1, Aptos por andar: 4, BWC Social: 1, C...",* Localizacao em ponto estrategico do bairro p...,"[3, (--), 1, 1, 110 m²]"


In [ ]:
anuncios.columns.tolist()

['titulo',
 'endereco',
 'condominio',
 'iptu',
 'aluguel',
 'itens_imovel',
 'descricao',
 'atributos']

In [ ]:
# 'titulo'
anuncios['titulo'] = anuncios['titulo'].str.strip()

# 'endereco'
anuncios['endereco'] = anuncios['endereco'].str.strip()
anuncios['bairro'] = anuncios['endereco'].apply(lambda x: x.split(' - ')[1])

# 'condominio'
anuncios['condominio'] = anuncios['condominio'].apply(lambda x: x if pd.isna(x) else np.nan if x == '' else x.replace('Cond. ','').replace('R$ ','').split(',')[0])

# 'iptu'
anuncios['iptu'] = anuncios['iptu'].apply(lambda x: x if pd.isna(x) else np.nan if x == '' else x.replace('IPTU  R$ ', '').split(',')[0]).astype('float64')

# 'aluguel'
anuncios['aluguel'] = anuncios['aluguel'].apply(lambda x: x.replace('R$ ','').replace('.','')).astype('float64')

# 'itens_imovel'

# 'descricao'

# formatando valores
anuncios['area'] = anuncios['atributos'].apply(lambda x: x[4]).replace(' m²',0)
anuncios['quartos'] = anuncios['atributos'].apply(lambda x: x[0]).replace('(--)',0)
anuncios['suites'] = anuncios['atributos'].apply(lambda x: x[1]).replace('(--)',0)
anuncios['banheiros'] = anuncios['atributos'].apply(lambda x: x[2]).replace('(--)',0)
anuncios['vagas'] = anuncios['atributos'].apply(lambda x: x[3]).replace('(--)',0)

In [ ]:
anuncios['area'] = anuncios['atributos'].apply(lambda x: x[4]).replace(' m²',0)
anuncios['quartos'] = anuncios['atributos'].apply(lambda x: x[0]).replace('(--)',0)
anuncios['suites'] = anuncios['atributos'].apply(lambda x: x[1]).replace('(--)',0)
anuncios['banheiros'] = anuncios['atributos'].apply(lambda x: x[2]).replace('(--)',0)
anuncios['vagas'] = anuncios['atributos'].apply(lambda x: x[3]).replace('(--)',0)

In [ ]:
anuncios

,titulo,endereco,condominio,iptu,aluguel,itens_imovel,descricao,atributos,bairro,quartos,suites,banheiros,vagas,area
0,Apartamento no CENTRO de 98 m2,"AL DOUTOR CARLOS DE CARVALHO, no 816 - Centro ...",1422,91.00,2300.0,"Andar: 6.o, Aptos por andar: 3, Central de gas...","Ponto de Referencia: Au Au Lanches, Hospital S...","[(--), 1, 1, 1, 98 m²]",Centro,(--),1,1,1,98 m²
1,Apartamento no Centro de 84 m2 com 2 quartos,"R MARIANO TORRES, no 401 - Centro - Curitiba",815,122.02,2600.0,"Aptos por andar: 4, BWC Social: 1, Central de ...",Pontos de Referencia: Mercadorama rua XV e ao ...,"[2, (--), 1, 1, 84 m²]",Centro,2,(--),1,1,84 m²
2,Apartamento no Centro de 48.6 m2 com 1 quarto,"R CONSELHEIRO LAURINDO, no 781 - Centro - Curi...",545,60.87,1650.0,"Aptos por andar: 6, BWC Social: 1, Central de ...","Pontos de Referencia: Terminal do Guadalupe, ...","[1, (--), 1, 1, 48.6 m²]",Centro,1,(--),1,1,48.6 m²
3,Apartamento no Tingui de 70.81 m2 com 3 quartos,"R NICOLAU SALOMAO, no 550 - Tingui - Curitiba",460,53.87,1700.0,"Aptos por andar: 4, BWC Social: 1, Condominio ...","Pontos de Referencia: Unidade de Saude Tingui,...","[3, (--), 1, 1, 70.81 m²]",Tingui,3,(--),1,1,70.81 m²
4,Apartamento no Centro de 106.31 m2 com 1 quarto,"R JOAO NEGRAO, no 805 - Centro - Curitiba",NaN,197.09,1500.0,"Aptos por andar: 2, BWC Social: 1, Central de ...","Pontos de Referencia: Shopping Estacao, UTFPR ...","[1, (--), 1, (--), 106.31 m²]",Centro,1,(--),1,(--),106.31 m²
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580,Apartamento no PortAGo de 46 mAe com 1 quarto,"Rua CapitAGo Tenente Maris de Barros, nAs 206 ...",240,65.53,1500.0,"Andar: 3, BWC Social: 1, Churrasqueiras: 1, Co...","Apartamento para locaASSAGo em Curitiba, no ba...","[1, (--), 1, (--), 46 mÂē]",PortAGo,1,(--),1,(--),46 mÂē
581,Apartamento no Bigorrilho de 104 m2 com 3 quartos,"Rua Padre Anchieta, no 2690 - Bigorrilho - Cur...",944,0.01,3000.0,"Andar: 2, BWC Social: 2, Churrasqueiras: 1, Co...",Apartamento tem:3 quartos (sendo um deles suit...,"[3, 1, 3, (--), 104 m²]",Bigorrilho,3,1,3,(--),104 m²
582,Apartamento no Bigorrilho de 188 m2 com 3 quartos,"Rua General Aristides Athayde Junior, no 1355 ...",1282.6,263.37,5000.0,"Andar: 3, BWC Social: 2, Condominio: Edificio ...","Apartamento para Locacao, 3 quartos sendo um s...","[3, 1, 3, 2, 188 m²]",Bigorrilho,3,1,3,2,188 m²
583,Apartamento no JUVEVE de 110 m2 com 3 quartos,"R MANOEL EUFRASIO, no 695 - Juveve - Curitiba",580,125.00,2000.0,"Andar: 1, Aptos por andar: 4, BWC Social: 1, C...",* Localizacao em ponto estrategico do bairro p...,"[3, (--), 1, 1, 110 m²]",Juveve,3,(--),1,1,110 m²
